<a target="_blank" href="https://colab.research.google.com/github/TransformerLensOrg/TransformerLens/blob/main/demos/Grokking_Demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Interpreting Addition Mod 3 Map

# Setup
(No need to read)

In [1]:
TRAIN_MODEL = True

In [2]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
import os

DEVELOPMENT_MODE = True
IN_GITHUB = os.getenv("GITHUB_ACTIONS") == "true"
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")

    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")
    ipython.magic("pip install ipympl")
    ipython.magic("pip install scipy")

if IN_COLAB or IN_GITHUB:
    %pip install transformer_lens
    %pip install circuitsvis
    %pip install ipympl

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_794282/1878011034.py:22: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_794282/1878011034.py:23: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")
/tmp/ipykernel_794282/1878011034.py:24: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("pip install ipympl")



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/tmp/ipykernel_794282/1878011034.py:25: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("pip install scipy")



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: notebook_connected


In [4]:
pio.templates['plotly'].layout.xaxis.title.font.size = 20
pio.templates['plotly'].layout.yaxis.title.font.size = 20
pio.templates['plotly'].layout.title.font.size = 30

In [5]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import os
import tqdm.auto as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [6]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache


device = "cuda" if torch.cuda.is_available() else "cpu"

/home/philippw/Stuff/programming/mechint/getting_started/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning:

CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)



Plotting helper functions:

In [7]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [8]:
# Define the location to save the model, using a relative path
PTH_LOCATION = "workspace/_scratch/local_optimimum_mod_3.pth"

# Create the directory if it does not exist
os.makedirs(Path(PTH_LOCATION).parent, exist_ok=True)

# Model Training

## Config

In [50]:
p = 3
frac_train = 1

# Optimizer config
lr = 1e-3
wd = 1e-2
betas = (0.9, 0.999)

num_epochs = 20000
checkpoint_every = 200

DATA_SEED = 598

## Define Task
* Define modular addition
* Define the dataset & labels

Input format:
|a|b|=|

In [51]:
a_vector = einops.repeat(torch.arange(p), "i -> (i j)", j=p)
b_vector = einops.repeat(torch.arange(p), "j -> (i j)", i=p)
equals_vector = einops.repeat(torch.tensor(p), " -> (i j)", i=p, j=p)


In [52]:
dataset = torch.stack([a_vector, b_vector, equals_vector], dim=1).to(device)
print(dataset)
print(dataset.shape)

tensor([[0, 0, 3],
        [0, 1, 3],
        [0, 2, 3],
        [1, 0, 3],
        [1, 1, 3],
        [1, 2, 3],
        [2, 0, 3],
        [2, 1, 3],
        [2, 2, 3]])
torch.Size([9, 3])


In [53]:
labels = (dataset[:, 0] + dataset[:, 1]) % p
print(labels.shape)
print(labels)

torch.Size([9])
tensor([0, 1, 2, 1, 2, 0, 2, 0, 1])


Convert this to a train + test set - 30% in the training set

In [54]:
torch.manual_seed(DATA_SEED)
indices = torch.randperm(p*p)
cutoff = int(p*p*frac_train)
train_indices = indices[:cutoff]
test_indices = indices[cutoff:]

train_data = dataset[train_indices]
train_labels = labels[train_indices]
test_data = dataset[test_indices]
test_labels = labels[test_indices]
print(train_data)
print(train_labels)
print(train_data.shape)
print(test_data[:5])
print(test_labels[:5])
print(test_data.shape)

tensor([[0, 2, 3],
        [2, 2, 3],
        [1, 0, 3],
        [0, 0, 3],
        [1, 2, 3],
        [0, 1, 3],
        [2, 1, 3],
        [1, 1, 3],
        [2, 0, 3]])
tensor([2, 1, 1, 0, 0, 1, 0, 2, 2])
torch.Size([9, 3])
tensor([], size=(0, 3), dtype=torch.int64)
tensor([], dtype=torch.int64)
torch.Size([0, 3])


## Define Model

In [55]:
cfg = HookedTransformerConfig(
    n_layers = 1,
    n_heads = 2,
    d_model = 3,
    d_head = 3,
    d_mlp = 0,
    act_fn = "relu",
    normalization_type=None,
    d_vocab=p+1,
    d_vocab_out=p,
    n_ctx=3,
    init_weights=True,
    device=device,
    seed = 997,
)
model = HookedTransformer(cfg)
for name, param in model.named_parameters():
    if "b_" in name:
        param.requires_grad = False

Disable the biases, as we don't need them for this task and it makes things easier to interpret.

## Define Optimizer + Loss

In [59]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, betas=betas)
def loss_fn(logits, labels):
    if len(logits.shape)==3:
        logits = logits[:, -1]
    logits = logits.to(torch.float64)
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[:, None])[:, 0]
    return -correct_log_probs.mean()
train_logits = model(train_data)
train_loss = loss_fn(train_logits, train_labels)
print(train_loss)
test_logits = model(test_data)
test_loss = loss_fn(test_logits, test_labels)
print(test_loss)
print("Uniform loss:")
print(np.log(p))

tensor(1.1591, dtype=torch.float64, grad_fn=<NegBackward0>)
tensor(nan, dtype=torch.float64, grad_fn=<NegBackward0>)


## Actually Train

**Weird Decision:** Training the model with full batch training rather than stochastic gradient descent. We do this so to make training smoother and reduce the number of slingshots.

In [68]:
train_losses = []
test_losses = []
model_checkpoints = []
checkpoint_epochs = []
if TRAIN_MODEL:
    for epoch in tqdm.tqdm(range(num_epochs)):
        train_logits = model(train_data)
        train_loss = loss_fn(train_logits, train_labels)
        train_loss.backward()
        train_losses.append(train_loss.item())

        optimizer.step()
        optimizer.zero_grad()

        with torch.inference_mode():
            test_logits = model(test_data)
            test_loss = loss_fn(test_logits, test_labels)
            test_losses.append(test_loss.item())

        if ((epoch+1)%checkpoint_every)==0:
            checkpoint_epochs.append(epoch)
            model_checkpoints.append(copy.deepcopy(model.state_dict()))
            print(f"Epoch {epoch} Train Loss {train_loss.item()} Test Loss {test_loss.item()}")

  0%|          | 0/20000 [00:00<?, ?it/s]

Epoch 199 Train Loss 8.44302405715638e-11 Test Loss nan
Epoch 399 Train Loss 8.439357854013343e-11 Test Loss nan
Epoch 599 Train Loss 8.435760731414049e-11 Test Loss nan
Epoch 799 Train Loss 8.432935830607325e-11 Test Loss nan
Epoch 999 Train Loss 8.430335441567806e-11 Test Loss nan
Epoch 1199 Train Loss 8.427670906309164e-11 Test Loss nan
Epoch 1399 Train Loss 8.424648632520385e-11 Test Loss nan
Epoch 1599 Train Loss 8.423222612724569e-11 Test Loss nan
Epoch 1799 Train Loss 8.422830333922623e-11 Test Loss nan
Epoch 1999 Train Loss 8.422632960940564e-11 Test Loss nan
Epoch 2199 Train Loss 8.422677369861628e-11 Test Loss nan
Epoch 2399 Train Loss 8.423244817185363e-11 Test Loss nan
Epoch 2599 Train Loss 8.423627227338336e-11 Test Loss nan
Epoch 2799 Train Loss 8.424017038978108e-11 Test Loss nan
Epoch 2999 Train Loss 8.42381226450924e-11 Test Loss nan
Epoch 3199 Train Loss 8.42260828931826e-11 Test Loss nan
Epoch 3399 Train Loss 8.422294959709201e-11 Test Loss nan
Epoch 3599 Train Loss 

In [69]:
torch.save(
    {
        "model":model.state_dict(),
        "config": model.cfg,
        "checkpoints": model_checkpoints,
        "checkpoint_epochs": checkpoint_epochs,
        "test_losses": test_losses,
        "train_losses": train_losses,
        "train_indices": train_indices,
        "test_indices": test_indices,
    },
    PTH_LOCATION)

In [70]:
if not TRAIN_MODEL:
    cached_data = torch.load(PTH_LOCATION)
    model.load_state_dict(cached_data['model'])
    model_checkpoints = cached_data["checkpoints"]
    checkpoint_epochs = cached_data["checkpoint_epochs"]
    test_losses = cached_data['test_losses']
    train_losses = cached_data['train_losses']
    train_indices = cached_data["train_indices"]
    test_indices = cached_data["test_indices"]

# Look at weights

## Embedding weights

In [73]:
from scipy.linalg import qr

print(model.embed.W_E.data)
print(model.embed.W_E.data.shape)

tensor = model.embed.W_E.data.cpu().numpy()
for i in range(tensor.shape[0]):
    print(f"Token {i}: {np.linalg.norm(tensor[i])}")

# Create a matrix with tensor[0] as the first column
ttr = tensor.transpose()
print(ttr)

# Perform QR decomposition to obtain an orthonormal matrix
Q, R = qr(ttr)

print("Q: ", Q)
print("R: ", R)
print(np.matmul(Q, R))

def get_angle(a, b):
    return np.arccos(
        np.dot(a, b)
        / (np.linalg.norm(a) * np.linalg.norm(b))
    ) / np.pi * 180

dot_product = np.dot(tensor[0], tensor[3])
print(dot_product, get_angle(tensor[0], tensor[3]))
product2 = np.dot(R[:, 0], R[:, 3])
print(product2, get_angle(R[:, 0], R[:, 3]))

tensor([[ 0.4255,  0.5214,  1.2031],
        [ 1.5470,  0.0481, -1.3921],
        [-2.2277,  0.0171,  0.2355],
        [-0.1093, -1.4294,  1.3221]])
torch.Size([4, 3])
Token 0: 1.37850821018219
Token 1: 2.081744432449341
Token 2: 2.240163564682007
Token 3: 1.9501159191131592
[[ 0.4254886   1.5470358  -2.227684   -0.10925768]
 [ 0.5213902   0.04806257  0.01714477 -1.4293535 ]
 [ 1.2030779  -1.3921316   0.23550649  1.3221059 ]]
Q:  [[-0.30865872  0.9055637   0.29100567]
 [-0.3782279   0.16386488 -0.91109383]
 [-0.8727391  -0.39128345  0.2919311 ]]
R:  [[-1.3785081   0.719283    0.47557402 -0.5795088 ]
 [ 0.          1.9535332  -2.1066506  -0.85047877]
 [ 0.          0.         -0.5951376   1.6564443 ]]
[[ 0.42548853  1.5470358  -2.227685   -0.10925758]
 [ 0.5213902   0.04806259  0.0171448  -1.4293534 ]
 [ 1.2030779  -1.3921316   0.2355063   1.3221059 ]]
0.7988577 72.71251367741992
0.79885757 72.71251367741992


In [1]:
import matplotlib.pyplot as plt
import numpy as n

%matplotlib widget

tensor = model.embed.W_E.data.cpu().numpy()

def plot_tensor(tensor):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Define the origin
    origin = np.zeros((4, 3))

    # Plot the arrows
    for i in range(tensor.shape[0]):
        ax.quiver(origin[i, 0], origin[i, 1], origin[i, 2], 
                tensor[i, 0], tensor[i, 1], tensor[i, 2])
        ax.text(tensor[i, 0], tensor[i, 1], tensor[i, 2], f"Embedding {i}", color='red')

    # Set labels
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xlim(tensor[:, 0].min(), tensor[:, 0].max())
    ax.set_ylim(tensor[:, 1].min(), tensor[:, 1].max())
    ax.set_zlim(tensor[:, 2].min(), tensor[:, 2].max())

plot_tensor(tensor)
plot_tensor(R.transpose())

plt.show()

NameError: name 'model' is not defined